In [1]:
using NeuralNetworkAnalysis, Plots
using NeuralNetworkAnalysis: read_nnet_mat

In [2]:
controller = []
for i=1:9
    path = "/home/sguadalupe/Documents/Julia/VCASnnets/"
    file = "VertCAS_noResp_pra0" * string(i) * "_v9_20HU_200.mat"
    push!(controller, read_nnet_mat(path * file, act_key="act_fcns"))
end

In [204]:
g = 32.2
#=
a = [[-g/8, 0, g/8],
      [-g/3, -7g/24, g/4],
      [g/4, 7g/24, g/3],
      [-g/3, -7g/24, -g/4],
      [g/4, 7g/24, g/3],
      [-g/3],
      [g/3],
      [-g/3],
      [g/3]]
=#
a = [0,
     -7g/24,
     7g/24,
     -7g/24,
     7g/24,
     -g/3,
     g/3,
     -g/3,
     g/3]

A = [1  -1;
     0  1.;]
h0 = Interval(-133, -129)
hdot0 = [-19.5,-22.5,-25.5, -28.5]

function forward_adv(X::Vector, adv::Int)
    u = forward(controller[adv], X)
    return argmax(u)
end

function new_hddot(X, adv)
    hdot = X[2]*60
    
    dict = Dict()
    
    dict[1] = x -> false # COC
    
    dict[2] = x -> x <= 0 # DNC
    
    dict[3] = x -> x >= 0 # DND
    
    dict[4] = x -> x <= -1500 # DES1500
    
    dict[5] = x -> x >= 1500 # CL1500
    
    dict[6] = x -> x <= -1500 # SDES1500
    
    dict[7] = x -> x >= 1500 # SCL1500
    
    dict[8] = x -> x >= -2500 # SDES2500
    
    dict[9] = x -> x >= 2500 # SCL2500
    
    comply = dict[adv](hdot)
    comply ? 0.0 : a[adv]
end

function VCAS!(X0::Vector{Float64}, adv, t0, k)
    X = X0
    sol = Vector{Vector{Float64}}(undef, k[end])
    hddot = a[adv]
    for i=k
        #println(hddot)
        b = [-0.5hddot, hddot]
        X = A * X + b
        sol[i] = [X[1], X[2], adv]
        adv′ = forward_adv(sol[i], adv)
        adv = adv′
        hddot = new_hddot(sol[i], adv)
    end
    return sol
end

VCAS! (generic function with 1 method)

In [205]:
X0 = [sample(h0)[1], hdot0[rand(1:end)]]
adv = 1
k = 1:10
t0 = 25

sol = VCAS!(X0, adv, t0, k)

10-element Array{Array{Float64,1},1}:
 [-108.0154146002993, -22.5, 1.0]
 [-85.5154146002993, -22.5, 1.0]
 [-63.015414600299295, -22.5, 1.0]
 [-40.515414600299295, -22.5, 1.0]
 [-18.015414600299295, -22.5, 1.0]
 [-0.21124793363262828, -13.108333333333333, 5.0]
 [12.897085399700703, -13.108333333333333, 1.0]
 [26.005418733034034, -13.108333333333333, 1.0]
 [39.11375206636737, -13.108333333333333, 1.0]
 [52.2220853997007, -13.108333333333333, 1.0]